## Titanic with Keras

In [1]:
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten, Dense, Activation, Dropout

Using TensorFlow backend.


## setup

In [24]:
DATA_HOME_DIR = "/home/tsu-nera/repo/kaggle/input/titanic/"
row_data = pd.read_csv(DATA_HOME_DIR + 'train.csv', index_col=0)
test_data = pd.read_csv(DATA_HOME_DIR + 'test.csv', index_col=0)

In [25]:
test_ind = test_data.index

data= row_data[['Survived','Pclass','Sex','Age','SibSp','Parch','Fare']].dropna()
t_data = test_data[['Pclass','Sex','Age','SibSp','Parch','Fare']]

In [71]:
Pclass = pd.get_dummies(data['Pclass'])
Pclass.columns=['1st','2nd','3rd']
Pclass = Pclass.drop('1st',axis=1)

tPclass = pd.get_dummies(t_data['Pclass'])
tPclass.columns=['1st','2nd','3rd']
tPclass = tPclass.drop('1st',axis=1)

Sex = pd.get_dummies(data['Sex'])
Sex = Sex.drop('male',axis=1)

tSex = pd.get_dummies(t_data['Sex'])
tSex = tSex.drop('male',axis=1)

def adult_kids(age):
    if age > 15:
        if age > 60:
            return  'older'
        else :
            return   'adult'
    else :
        if age < 16:
            return   'kids'
        else :
            return np.nan

data['adult_kids'] = data['Age'].apply(adult_kids)
Age_cat = pd.get_dummies(data['adult_kids']).drop('adult',axis=1)
t_data['adult_kids'] = t_data['Age'].apply(adult_kids)
tAge_cat = pd.get_dummies(t_data['adult_kids']).drop('adult',axis=1)

Data_tmp = data[['Survived','SibSp','Parch','Fare']]

Merge_data = pd.merge(Data_tmp,Pclass,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,Sex,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,Age_cat,right_index=True,left_index=True)
y = Merge_data['Survived'].values
x = Merge_data.drop('Survived',axis=1).values

Data_tmp = t_data[['SibSp','Parch','Fare']]

Merge_data = pd.merge(Data_tmp,tPclass,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,tSex,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,tAge_cat,right_index=True,left_index=True)
tx = Merge_data

(x.shape, y.shape, tx.shape)

/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


((714, 8), (714,), (418, 8))

## Build Model

In [105]:
# create model
model = Sequential()
model.add(Dense(64, input_shape=(8,)))

for i in range(0, 8):
    model.add(Dense(units=64))
    model.add(Activation('relu'))

model.add(Dense(units=1))
model.add(Activation('linear'))

model.compile(loss='mean_squared_error', optimizer='rmsprop')

## Training

In [102]:
model.fit(x, y, epochs=300)

Epoch 1/300
714/714 [==============================] - 1s - loss: 0.2937     
Epoch 2/300
714/714 [==============================] - 0s - loss: 0.1910     
Epoch 3/300
714/714 [==============================] - 0s - loss: 0.1814     
Epoch 4/300
714/714 [==============================] - 0s - loss: 0.1698     
Epoch 5/300
714/714 [==============================] - 0s - loss: 0.1719     
Epoch 6/300
714/714 [==============================] - 0s - loss: 0.1747     
Epoch 7/300
714/714 [==============================] - 0s - loss: 0.1625     
Epoch 8/300
714/714 [==============================] - 0s - loss: 0.1649     
Epoch 9/300
714/714 [==============================] - 0s - loss: 0.1669     
Epoch 10/300
714/714 [==============================] - 0s - loss: 0.1726     
Epoch 11/300
714/714 [==============================] - 0s - loss: 0.1779     
Epoch 12/300
714/714 [==============================] - 0s - loss: 0.1851     
Epoch 13/300
714/714 [==============================] - 0s - 

714/714 [==============================] - 0s - loss: 0.1535     
Epoch 207/300
714/714 [==============================] - 0s - loss: 0.1456     
Epoch 208/300
714/714 [==============================] - 0s - loss: 0.1518     
Epoch 209/300
714/714 [==============================] - 0s - loss: 0.1413     
Epoch 210/300
714/714 [==============================] - 0s - loss: 0.1437     
Epoch 211/300
714/714 [==============================] - 0s - loss: 0.1423     
Epoch 212/300
714/714 [==============================] - 0s - loss: 0.1417     
Epoch 213/300
714/714 [==============================] - 0s - loss: 0.1406     
Epoch 214/300
714/714 [==============================] - 0s - loss: 0.1454     
Epoch 215/300
714/714 [==============================] - 0s - loss: 0.1388     
Epoch 216/300
714/714 [==============================] - 0s - loss: 0.1414     
Epoch 217/300
714/714 [==============================] - 0s - loss: 0.1467     
Epoch 218/300
714/714 [==============================]

## Testing

In [103]:
p_survived = model.predict_classes(tx.values)

 32/418 [=>............................] - ETA: 5s

In [104]:
submission = pd.DataFrame()
submission['PassengerId'] = test_ind
submission['Survived'] = p_survived

In [96]:
submission.to_csv('submission.csv', index=False)